<a href="https://colab.research.google.com/github/SSGrady/Rec-System/blob/main/Recommendation_System_StevenGrady.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! curl https://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1612k      0 --:--:-- --:--:-- --:--:-- 1613k


##Movie Recommendation System: Collaborative Filtering + Kmeans + Matrix Factorization

In [ ]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
  zip_ref.extractall('data')

In [26]:
import pandas as pd
#import the dataset

# dimensions of movies are (9742, 3)
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
# dimensions of ratings are (100836, 3)
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [24]:
# View movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [25]:
#View ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
